In [ ]:
import recordlinkage as rl
import pandas as pd
import os
import random as ran
from sklearn.model_selection import GridSearchCV

In [ ]:
schema_path = "../schema_matching/csv/schema_final.csv"

In [ ]:
schema = pd.read_csv(schema_path, index_col=0)
schema = schema.reset_index(drop=True) # Per evitare che gli indici siano letti come decimanli (i.e. 1.0 invece di 1)

# Preprocessamento

In [ ]:
from recordlinkage.preprocessing import *

## Cleaning dei dati ...

In [ ]:
schema_cleaned = schema # creo una copia per non modificare lo schema iniziale
schema_cleaned_path = "./csv/schema_cleaned.csv"

In [ ]:
# Cleaning dei dati iterando per riga
for i in range(len(schema.index)):
    row = schema.iloc[i]
    row_series = row.squeeze() # BOO
    nome = pd.Series(row["name"])
    try:
        nome_cleaned = clean(nome, lowercase=True, replace_by_none='\\n', replace_by_whitespace='\ {2,}', strip_accents='unicode', remove_brackets=False, encoding='utf-8', decode_error='strict')
    except:
        print(nome)
    # Si evita il cleaning di caratteri relative alle valute presenti nei datasets
    row_series_cleaned = clean(row_series, lowercase=True, replace_by_none=r'[^ \\.\\-\\_A-Za-z0-9$€£¥₩₽₹฿₪¤¢₺₱]+', replace_by_whitespace=r'[\\-\\_]', strip_accents=None, remove_brackets=False, encoding='utf-8', decode_error='strict')
    schema_cleaned.iloc[i] = row_series_cleaned
    schema_cleaned.iloc[i]["name"] = nome_cleaned[0]

In [ ]:
schema_cleaned.to_csv(schema_cleaned_path, index_label="id")

# Occorrenze token per riga

In [ ]:
# value_occurence(schema)

# Blocking

In [ ]:
indexer = rl.Index()
indexer.block(left_on='name', right_on='name')
candidate_links = indexer.index(schema_cleaned)
print(candidate_links)

# Labeling

In [ ]:
keys = ['name', 'country', 'market cap', 'founded year', 'employees', 'industry', 'sector',
     'ceo', 'revenue', 'stock', 'share price', 'city', 'address', 'website']

clear = lambda: os.system('clear')

In [ ]:
n_sampled_matches = 30000
choices = ran.choices(candidate_links, k=n_sampled_matches)

lKeys = {key : f"l_{key}" for key in keys}
rKeys = {key : f"r_{key}" for key in keys}

idxKeys = ["id_1", "id_2"]

choices_column = [*idxKeys, *lKeys.values(), *rKeys.values()] # id_1, id_2, l_attr, r_attr
choices_df = pd.DataFrame(columns=choices_column) # 
for choice in choices:
    lRow = schema_cleaned.iloc[[choice[0]]]
    rRow = schema_cleaned.iloc[[choice[1]]]

    lRow = lRow.rename(columns=lKeys).reset_index(drop=True)
    rRow = rRow.rename(columns=rKeys).reset_index(drop=True)

    idxs = pd.DataFrame({idxKeys[0]: [choice[0]], idxKeys[1]: [choice[1]]})
    row = pd.concat([idxs, lRow, rRow], axis=1)
    choices_df = pd.concat([choices_df, row], axis=0)

# choices_df.reset_index(inplace=True)
choices_df.set_index(idxKeys)

In [ ]:
choices_path = "./csv/choices.csv"
choices_df.to_csv(choices_path, index=None)

In [ ]:
choices_path_gallo = "./csv/gallo.csv"
choices_path_gatto = "./csv/gatto.csv"
choices_path_moli = "./csv/moli.csv"

sampled_matches_per_person = n_sampled_matches//3

choices_df_gallo = choices_df.iloc[:sampled_matches_per_person]
print(len(choices_df_gallo))

choices_df_gatto = choices_df.iloc[sampled_matches_per_person:2*sampled_matches_per_person]
print(len(choices_df_gatto))

choices_df_moli = choices_df.iloc[2*sampled_matches_per_person:]
print(len(choices_df_moli))

choices_df_gallo.to_csv(choices_path_gallo, index=None)
choices_df_gatto.to_csv(choices_path_gatto, index=None)
choices_df_moli.to_csv(choices_path_moli, index=None)

# Comparing

In [ ]:
compare_cl = rl.Compare()
compare_cl.string("name", "name", threshold=0.85, label="name")
compare_cl.string("country", "country", label="country")
compare_cl.string("founded year", "founded year", label="founded year")
compare_cl.string("industry", "industry", label="industry")
compare_cl.string("sector", "sector", label="sector")
compare_cl.string("address", "address", label="address")
compare_cl.string("city", "city", label="city")
compare_cl.string("ceo", "ceo", label="ceo")
features = compare_cl.compute(candidate_links, schema, schema)

In [ ]:
features

In [ ]:
features.describe()

In [ ]:
features.sum(axis=1).value_counts().sort_index(ascending=False)

In [ ]:
record_linked = features[features.sum(axis=1) > 1]

In [ ]:
print(record_linked)

# Caricamento dei dataset per classificazione

In [ ]:
golden_path_gallo = "./csv/golden_links_gallo.csv"
golden_path_gatto = "./csv/golden_links_gatto.csv"
golden_path_moli = "./csv/golden_links_moli.csv"

golden_gallo = pd.read_csv(golden_path_gallo, index_col=None)
golden_gatto = pd.read_csv(golden_path_gatto, index_col=None)
golden_moli = pd.read_csv(golden_path_moli, index_col=None)

golden = pd.concat([golden_gallo, golden_gatto, golden_moli], axis=0)

golden_path = "./csv/golden_links.csv"

golden = golden.reset_index(drop=True)
golden.to_csv(golden_path, index=None)

In [ ]:
golden = pd.read_csv(golden_path, index_col=[0,1])
golden_links = golden[golden["label"]==1].index

# Classification (Unsupervised)

In [ ]:
ECM = rl.ECMClassifier(binarize=0.6)
result = ECM.fit_predict(record_linked)
print("Possible record linked size:", len(record_linked))
print("Record linked size:", len(result))
print(f"Removed {len(record_linked) - len(result)} possible links")

# Evaluation (Unsupervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, result)

### Precision

In [ ]:
rl.precision(golden_links, result)

### Recall

In [ ]:
rl.recall(golden_links, result)

### Accuracy

In [ ]:
rl.accuracy(golden_links, result)

### F-score

In [ ]:
rl.fscore(golden_links, result)

### Specificity

In [ ]:
rl.specificity(golden_links, result)

# Classification (Supervised)

In [ ]:
rlSvm = rl.SVMClassifier()
params = {}
gridSearch = GridSearchCV(estimator=rlSvm, param_grid=params)

# Evaluation (Supervised)

### Confusion matrix

In [ ]:
rl.confusion_matrix(golden_links, )

### Precision

In [ ]:
rl.precision(golden_links,)

### Recall

In [ ]:
rl.recall(golden_links,)

### Accuracy

In [ ]:
rl.accuracy(golden_links,)

### F-score

In [ ]:
rl.fscore(golden_links,)

### Specificity

In [ ]:
rl.specificity(golden_links,)